<a href="https://colab.research.google.com/github/Sahel-Eskandar/Text-Classification-Sentimental-Analysis/blob/main/cleaning_messy_data_extract_date_with_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import regex as re
doc = []
with open('/content/drive/MyDrive/Coursera/course4/assignments/course4_assignment1/assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df_cleaned = pd.DataFrame(df,columns=['dates']) 

In [27]:
def date_sorter():
    doc = []
    with open('/content/drive/MyDrive/Coursera/course4/assignments/course4_assignment1/assets/dates.txt') as file:
      for line in file:
        doc.append(line)
    df = pd.Series(doc)
    # Your code here
    df_cleaned = pd.DataFrame(df,columns=['dates']) 
    # extract dates from original dataframe and saved into 3 new cols, incl a) any digit of 'days' b) any digit of 'month' and c) any digit with 4 repetitons of 'year'
    # e.g., 04/20/2009
    pat1 = r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))'
    clean_date = df.str.extractall(pat1)
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    # extract and append a) any digit of 'month' b) days e.g., 11/29 and c) two digit of 'year' e.g., 89/90 
    # e.g., 6/2008; 12/2009
    pat2 = r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'
    clean_date = pd.concat([clean_date,df[messay_index].str.extractall(pat2)])
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    # extract and append a) any digit of 'day' b) at least 3 characters of 'month' e.g., JAN/feb and b) any digit with 4 repetitons of 'year'
    # e.g., 20 Mar 2009
    pat3 = r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))'
    
    clean_date = pd.concat([clean_date,df[messay_index].str.extractall(pat3)])
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    # extract and append a) at least 3 characters of 'month' e.g., JAN/feb b) any dight of 'day' plus 'th/nd/st' and c) any digit with 4 repetitons of 'year'
    # Mar 20th, 2009
    pat4 = r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))'
    clean_date = pd.concat([clean_date,df[messay_index].str.extractall(pat4)])
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    del clean_date[3]
    del clean_date[4]
    
    # if the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009)
    pat5 = r'(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))'
    dates_without_day = df[messay_index].str.extractall(pat5)

    pat6 = r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))'
    dates_without_day =  pd.concat([dates_without_day, df[messay_index].str.extractall(pat6)])
    dates_without_day['day'] = 1
    clean_date =  pd.concat([clean_date, dates_without_day])
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    # Only year
    pat7 = r'(?P<origin>(?P<year>\d{4}))'
    dates_only_year = df[messay_index].str.extractall(pat7)
    dates_only_year['day'] = 1
    dates_only_year['month'] = 1
    clean_date = pd.concat([clean_date,dates_only_year])
    messay_index = ~df.index.isin([x[0] for x in clean_date.index])

    # normalized year
    clean_date['year'] = clean_date['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    clean_date['year'] = clean_date['year'].apply(lambda x: str(x))

    # normalized month
    clean_date['month'] = clean_date['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 1})
    clean_date.replace({"month": month_dict}, inplace=True)
    clean_date['month'] = clean_date['month'].apply(lambda x: str(x))

    # Day
    clean_date['day'] = clean_date['day'].apply(lambda x: str(x))

    # Cleaned date
    clean_date['date'] = clean_date['month'] + '/' + clean_date['day'] + '/' + clean_date['year']
    clean_date['index'] = clean_date['date'].droplevel('match').index
    clean_date['date'] = pd.to_datetime(clean_date['date'])
    return pd.Series(list(clean_date.sort_values(["date", "index"])['index'].values))

In [28]:
%%time
date_sorter()

CPU times: user 74.9 ms, sys: 3.67 ms, total: 78.5 ms
Wall time: 81.1 ms


0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [44]:
r1= r'(\d?\d[-|/]\d?\d[/|-]\d\d\d\d)' 
r2= r'(\d?\d[-|/]\d?\d[/|-]\d\d)'
r4= r'([A-Z][a-z]+\W?\s?\d\d\D?\D?\W?\s\d\d+)'
r5= r'([A-Z][a-z]+W?-?\d\d\D?\D?\W?\s\d\d+)'
r6= r'(\d\d\s[A-Za-z]+\s?\d\d\d\d)'
r7= r'([A-Z][a-z]+\W?\s\d\d\d\d)'
r8= r'(\d?\d[/|-]\d\d\d\d)'
r9= r'(\d\d\d\d)'
regexprs=[r1,r2,r4,r5,r6,r7,r8,r9]
doc = []
with open('/content/drive/MyDrive/Coursera/course4/assignments/course4_assignment1/assets/dates.txt') as file:
      for line in file:
        doc.append(line)
df = pd.Series(doc)
df3 = pd.DataFrame(df,columns=['dates']) 

def date_sorter1():
    for i in range(500):
        for regex in regexprs:
            a=re.findall(regex,df3.loc[i,'dates'])
            if len(a)>0:
                if regex == r2:
                  a[0] = a[0][:-2]+'19'+a[0][-2:]
                if regex == r8:
                  a[0] = a[0][:-4]+'01/'+a[0][-4:]
                if regex == r9:
                  a[0] = '01/01/'+a[0]
                try:
                  #print(i,regex,a, pd.Timestamp(a[0]))
                  df3.loc[i,'Extracted_date']= pd.Timestamp(a[0])
                except:
                  df3.loc[i,'Extracted_date']=pd.Timestamp(a[0].split()[-1])
                break
            else:
                continue
  
    return pd.Series(df3["Extracted_date"].reset_index(drop=False).sort_values(["Extracted_date", "index"]).index)

In [45]:
%%time
date_sorter1()

CPU times: user 150 ms, sys: 708 µs, total: 151 ms
Wall time: 155 ms


0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [43]:
df[date_sorter2() != date_sorter1()]

81    . Wile taking additional history, pt endorsed ...
82    Death of mother; 7/07/1974 Meaningful activiti...
83    09/14/2000 CPT Code: 90792: With medical servi...
84          5/18/71 Total time of visit (in minutes):\n
85    8/09/1981 Communication with referring physici...
86       6/05/93 CPT Code: 90791: No medical services\n
87    )Dilantin (PHENYTOIN) 100 MG CAPSULE Take 2 Ca...
88            12/8/82 Audit C=3Audit C Score Current:\n
89       8/26/89 CPT Code: 90791: No medical services\n
90      10/13/95 CPT Code: 90791: No medical services\n
91    4/19/91 Communication with referring physician...
92    .APS - Psychiatry consult paged/requested in p...
93    9/20/76 CPT Code: 90801 - Psychiatric Diagnosi...
94                     12/08/1990 @11 am [report_end]\n
dtype: object

In [22]:
from datetime import datetime

def date_sorter2():
    
    parsed_dates = dict.fromkeys(df.index)
    
    all_numeric = df[:125]
    for (ind, date) in zip(all_numeric.index, all_numeric.str.findall(r"[\d]{1,2}[/-]{1}[\d]{1,2}[/-]{1}[\d]{2,4}")):

        _date = date[0]

        if "-" not in _date:
            if len(_date.split("/")[2]) < 4:      
                _strdate = _date[:-2] + "19" + _date[-2:]

                parsed_dates[ind] = datetime.strptime(_strdate, "%m/%d/%Y")
            else:

                parsed_dates[ind] = datetime.strptime(_date, "%m/%d/%Y")
                
        elif "-" in _date:
            if len(_date.split("-")[2]) < 4:
                _strdate = _date[:-2] + "19" + _date[-2:]

                parsed_dates[ind] = datetime.strptime(_strdate, "%m-%d-%Y")
            else:

                parsed_dates[ind] = datetime.strptime(_date, "%m-%d-%Y")
    
    month_in_words = df[125:194]
    for (ind, date) in zip(month_in_words.index, month_in_words.str.findall(r"[\d]{1,2} [\w]{3,} [\d]{2,4}")):
            # parsed_dates[ind] = date[0]
            _strdate  = date[0]
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
            except ValueError:
                try:
                    parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
                except:
                    print("FUCK")
                    
    start_with_month = df[194:228]
    for (ind, date) in zip(start_with_month.index, start_with_month.str.findall(r"[\w]{3,}[.,]* [\d]+[,]* [\d]{2,4}")):
            # parsed_dates[ind] = date[0]
            _strdate = date[0].replace(",", "").replace(".", "")
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%B %d %Y")
            except ValueError:
                try:
                    parsed_dates[ind] = datetime.strptime(_strdate, "%b %d %Y")
                except:
                    print("FUCK")
    
    month_and_year_only = df[228:343]
    for (ind, date) in zip(month_and_year_only.index, month_and_year_only.str.findall(r"[A-Z]{1}[a-z]{2,}[,]* [\d]{4}")):
        # parsed_dates[ind] = "01 " + date[0]
        _strdate = "01 " + date[0].replace(",", "").replace("Janaury", "January").replace("Decemeber", "December")
        try:
            parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
        except ValueError:
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
            except:
                print("FUCK", _strdate)
    
    month_and_year_numeric = df[343:455]
    for (ind, date) in zip(month_and_year_numeric.index, month_and_year_numeric.str.findall(r"[\d]{1,2}/[\d]{2,4}")):
        # parsed_dates[ind] = "01/" + date[0]
        _strdate = "01/" + date[0]
        parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")
        
    year_only = df[455:]
    for (ind, date) in zip(year_only.index, year_only.str.findall(r"[\d]{4}")):
        # parsed_dates[ind] =  "01/01/" + date[0]
        _strdate = "01/01/" + date[0]
        parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")
        
    dates = pd.Series(parsed_dates, name = "date")
    return pd.Series(dates.reset_index(drop = False).sort_values(["date", "index"]).index)

In [23]:
%%time 
date_sorter2()

CPU times: user 19.6 ms, sys: 0 ns, total: 19.6 ms
Wall time: 19.6 ms


0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [33]:
df[date_sorter2() != date_sorter1()]

81    . Wile taking additional history, pt endorsed ...
82    Death of mother; 7/07/1974 Meaningful activiti...
83    09/14/2000 CPT Code: 90792: With medical servi...
84          5/18/71 Total time of visit (in minutes):\n
85    8/09/1981 Communication with referring physici...
86       6/05/93 CPT Code: 90791: No medical services\n
87    )Dilantin (PHENYTOIN) 100 MG CAPSULE Take 2 Ca...
88            12/8/82 Audit C=3Audit C Score Current:\n
89       8/26/89 CPT Code: 90791: No medical services\n
90      10/13/95 CPT Code: 90791: No medical services\n
91    4/19/91 Communication with referring physician...
92    .APS - Psychiatry consult paged/requested in p...
93    9/20/76 CPT Code: 90801 - Psychiatric Diagnosi...
94                     12/08/1990 @11 am [report_end]\n
dtype: object